# STEP 0: Install dependencies

In [15]:
# installing Google's AI (Gemini), LangGraph, and LangChain – tools that help talk to AI models.

#like installing WhatsApp before you can chat.
#google-generativeai==0.8.5    This is Google's official Python SDK for using Gemini AI models.  It allows you to generate text, chat, and use multimodal capabilities.
#google-ai-generativelanguage==0.6.15   This is a lower-level client for the Generative Language API (used under the hood by google-generativeai).  Handles actual API interactions with Gemini models.

In [1]:
!pip install -qU google-generativeai==0.8.5 google-ai-generativelanguage==0.6.15 langgraph langchain langchain-google-genai openai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.1/757.1 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 18.5 MB/s eta 0:00:00


# STEP 1: Imports and secure API key input

In [2]:
import os
import getpass
from langgraph.graph import StateGraph, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage


In [ ]:
#os and getpass: Used to securely ask for your API key (like your Google Gemini password).

#StateGraph: Used to draw your flowchart or logic graph – it helps track steps.

#ChatGoogleGenerativeAI: Lets you talk to Google's Gemini AI.

#HumanMessage: Helps send messages to AI like “I have chest pain.”




# Secure Gemini API Key input

In [3]:


os.environ['GOOGLE_API_KEY'] = getpass.getpass("Enter your Gemini API key: ")


Enter your Gemini API key: ··········


In [ ]:
#API key is like a password that gives your code permission to talk to Google's AI.

# STEP 2: Initialize Gemini 1.5 Flash

In [4]:
llm = ChatGoogleGenerativeAI(model = "models/gemini-1.5-flash-latest", temperature =0.2)


In [ ]:

#ChatGoogleGenerativeAI(...)	This is a constructor from LangChain (or similar), initializing a chat model using Gemini.
#model="models/gemini-1.5-flash-latest"	Specifies which Gemini model to use — in this case, Gemini 1.5 Flash (latest).
#temperature=0.2	Controls creativity or randomness of the response. Lower = more deterministic, higher = more creative.

# STEP 3: Node to ask user for symptom

In [5]:
def get_symptom(state: dict) ->dict:
  symptom = input("Welcome to XYZ hospital, Please enter your symptom")
  state["symptom"] = symptom
  return state

# STEP 4: Node to classify the symptom

In [6]:
def classify_symptom(state: dict) -> dict:
  prompt = (
      "You are a helpful Medical Assistant, Classify the symptoms below into one of the categories \n"
      "-General\n -Emergency \n -mental health \n"
      f"Symptom : {state['symptom']} \n"
      "Respond only with one word : General, Emergency Or Mental Health"
      "#Example : input : I have fever, Output : General"
  )
  response = llm.invoke([HumanMessage(content=prompt)])
  category = response.content.strip()
  print(f"LLM classifies the symptom as : {category}")  #debug
  state["category"] = category
  return state

In [ ]:
#  response = llm.invoke([HumanMessage(content=prompt)])
#You're sending the prompt to Gemini using llm.invoke().
#HumanMessage(content=prompt) is how you talk to Gemini using LangChain.
#Gemini reads the message and sends back an answer like "Emergency".

#  category = response.content.strip()
#You take the text response from Gemini.
#.strip() just removes extra spaces from the beginning or end.
#For example: " Emergency " becomes "Emergency".

# STEP 5: Router logic to route to the correct node


In [7]:
def symptom_router(state:dict) -> dict:
  cat = state["category"].lower()   #General, general, Mental mental
  if "general" in cat:
    return"general"
  elif "emergency" in cat:
    return"emergency"
  elif "mental" in cat:
    return"mental_health"
  else:
    return"general"


In [ ]:
#  cat = state["category"].lower()
#This gets the category from the state dictionary.
#Example: "Emergency" or "Mental Health"
#.lower() converts it to lowercase → "emergency", "mental health"
#This helps in comparing without worrying about capital letters.

# STEP 6: Category-specific response nodes


In [8]:
def general_node(state: dict) -> dict:
  state["answer"] = f" '{state['symptom']}' : seems general : directing you to general ward for consulting a doctor"
  return state

def emergency_node(state:dict) -> dict:
  state["answer"] =f" ' {state['symptom']}': It is a Medical Emergency : seeking immediate help"
  return state

def mental_health_node(state: dict) -> dict:
  state["answer"] = f" '{state['symptom']}: seems like a medical health issue: talk to our counsellor"
  return state

# STEP 7: Build LangGraph


In [9]:
builder = StateGraph(dict)

#define the nodes
builder.set_entry_point("get_symptom")
builder.add_node("get_symptom", get_symptom)
builder.add_node("classify", classify_symptom)
builder.add_node("general", general_node)
builder.add_node("emergency", emergency_node)
builder.add_node("mental_health", mental_health_node)

builder.add_edge("get_symptom", "classify")
builder.add_conditional_edges("classify", symptom_router, {
    "general": "general",
    "emergency": "emergency",
    "mental_health": "mental_health"
})

builder.add_edge("general", END)
builder.add_edge("emergency", END)
builder.add_edge("mental_health", END)



In [ ]:
#Tells LangGraph:

#Where to start (get_symptom)

#What happens after that (classify, route, etc.)

#Where the program ends (END)

# STEP 8: Compile and invoke the graph


In [10]:
graph = builder.compile()


In [14]:
final_state = graph.invoke({})
print("final Output \n")
print(final_state["answer"])

Welcome to XYZ hospital, Please enter your symptomi have chest pain
LLM classifies the symptom as : Emergency
final Output 

 ' i have chest pain': It is a Medical Emergency : seeking immediate help
